# Structural Lib IS 456 - Colab Demo (v0.13.0)

This notebook demonstrates the complete workflow:
1. **Design** - Beam flexure and shear design per IS 456
2. **Outputs** - BBS, DXF drawings, HTML reports
3. **Insights** - Precheck, sensitivity analysis, constructability scoring (NEW in v0.13!)

In [ ]:
!pip -q install "structural-lib-is456[dxf]"


: 

In [ ]:
from structural_lib import api
print('Version:', api.get_library_version())


In [ ]:
from structural_lib import flexure

res = flexure.design_singly_reinforced(
    b=300, d=450, d_total=500, mu_knm=150, fck=25, fy=500
)
print('Ast required (mm^2):', round(res.ast_required))
print('Status:', 'OK' if res.is_safe else res.error_message)


In [ ]:
import json

job = {
    'schema_version': 1,
    'code': 'IS456',
    'units': 'IS456',
    'job_id': 'colab_job_001',
    'beam': {
        'b_mm': 230,
        'D_mm': 500,
        'd_mm': 450,
        'd_dash_mm': 50,
        'fck_nmm2': 25,
        'fy_nmm2': 500,
        'asv_mm2': 100,
    },
    'cases': [
        {'case_id': 'LC1', 'mu_knm': 80, 'vu_kn': 55},
        {'case_id': 'LC2', 'mu_knm': 110, 'vu_kn': 75},
        {'case_id': 'LC3', 'mu_knm': 140, 'vu_kn': 95},
    ],
}

with open('job.json', 'w', encoding='utf-8') as f:
    json.dump(job, f, indent=2)

print('Wrote job.json')


In [ ]:
!python -m structural_lib job job.json -o ./job_out
!python -m structural_lib critical ./job_out --top 5 --format=csv -o critical.csv
!python -m structural_lib report ./job_out --format=html -o report.html


In [ ]:
from IPython.display import HTML, IFrame, display
from pathlib import Path
import pandas as pd

display(HTML(Path('report.html').read_text(encoding='utf-8')))
pd.read_csv('critical.csv').head(5)


In [ ]:
# If inline HTML does not render, use an iframe:
IFrame('report.html', width=900, height=600)


## BBS + DXF + Mark Consistency (Optional)

This section generates a small CSV, runs design → BBS → DXF, then checks
that bar marks match between the CSV and DXF callouts.


In [ ]:
import csv

rows = [{
    'BeamID': 'B1',
    'Story': 'G',
    'b': 230,
    'D': 500,
    'Span': 4000,
    'Cover': 40,
    'fck': 25,
    'fy': 500,
    'Mu': 120,
    'Vu': 80,
    'Ast_req': 0,
    'Asc_req': 0,
    'Stirrup_Dia': 8,
    'Stirrup_Spacing': 150,
}]

with open('beams_small.csv', 'w', newline='') as f:
    w = csv.DictWriter(f, fieldnames=rows[0].keys())
    w.writeheader()
    w.writerows(rows)

print('Wrote beams_small.csv')


In [ ]:
!python -m structural_lib design beams_small.csv -o results_small.json
!python -m structural_lib bbs results_small.json -o schedule_small.csv
!python -m structural_lib dxf results_small.json -o drawings_small.dxf
!python -m structural_lib mark-diff --bbs schedule_small.csv --dxf drawings_small.dxf --format json -o mark_diff.json


In [ ]:
import json
import pandas as pd
from IPython.display import display

with open('mark_diff.json', 'r', encoding='utf-8') as f:
    mark_diff = json.load(f)

print(mark_diff)
display(pd.read_csv('schedule_small.csv').head(5))


In [ ]:
from pathlib import Path

try:
    import ezdxf
    from ezdxf.addons import drawing
    from ezdxf.addons.drawing.matplotlib import MatplotlibBackend
    import matplotlib

    matplotlib.use('Agg')
    import matplotlib.pyplot as plt

    doc = ezdxf.readfile('drawings_small.dxf')
    msp = doc.modelspace()

    fig = plt.figure()
    ax = fig.add_axes([0, 0, 1, 1])
    ax.set_aspect('equal')
    ax.axis('off')

    ctx = drawing.RenderContext(doc)
    backend = MatplotlibBackend(ax)
    drawing.Frontend(ctx, backend).draw_layout(msp)
    backend.finalize()
    ax.autoscale()

    out_path = Path('drawings_small.png')
    fig.savefig(out_path, dpi=200, bbox_inches='tight', pad_inches=0.1)
    plt.close(fig)
    print(f'Rendered {out_path}')
except Exception as exc:
    print('Render skipped:', exc)


In [ ]:
from IPython.display import Image, display
from pathlib import Path

if Path('drawings_small.png').exists():
    display(Image('drawings_small.png'))


## Insights Module (v0.13+) - Smart Features

**✅ Available in v0.13.0+** - Install with: `!pip install -U "structural-lib-is456[dxf]>=0.13.0"`

The insights module provides three advisory features:
1. **Quick Precheck** - Heuristic validation before full design (~1ms)
2. **Sensitivity Analysis** - Identify critical parameters and robustness
3. **Constructability Scoring** - Ease of construction assessment (0-100 scale)

In [ ]:
# Example 1: Quick Precheck (Advisory - catches likely failures before full design)
from structural_lib.insights import quick_precheck

precheck_result = quick_precheck(
    span_mm=6000,
    b_mm=300,
    d_mm=400,  # Might be too shallow for 6m span
    D_mm=450,
    mu_knm=140,
    fck_nmm2=25,
    fy_nmm2=500
)

print(f"Risk Level: {precheck_result.risk_level}")
print(f"Execution Time: {precheck_result.check_time_ms:.2f} ms")
print(f"\nWarnings ({len(precheck_result.warnings)}):")
for w in precheck_result.warnings:
    emoji = '🔴' if w.severity == 'ERROR' else '🟡' if w.severity == 'WARNING' else 'ℹ️'
    print(f"  {emoji} {w.message}")
    print(f"     Suggestion: {w.suggestion}")

In [ ]:
# Example 2: Sensitivity Analysis (Identify critical parameters)
from structural_lib.insights import sensitivity_analysis
from structural_lib.api import design_beam_is456

# Define baseline parameters
params = {
    "units": "IS456",
    "mu_knm": 120.0,
    "vu_kn": 80.0,
    "b_mm": 300.0,
    "D_mm": 500.0,
    "d_mm": 450.0,
    "fck_nmm2": 25.0,
    "fy_nmm2": 500.0,
}

# Run sensitivity analysis
sensitivities, robustness = sensitivity_analysis(
    design_function=design_beam_is456,
    base_params=params,
    parameters_to_vary=["d_mm", "b_mm", "fck_nmm2", "fy_nmm2"]
)

print("Sensitivity Analysis Results (Ranked by Impact):")
print("=" * 70)
for s in sensitivities:
    direction = "↑ increases" if s.sensitivity > 0 else "↓ decreases"
    impact_pct = abs(s.sensitivity) * 10  # For 10% parameter change
    print(f"{s.parameter:12s}: S={s.sensitivity:+.3f} [{s.impact.upper():8s}]")
    print(f"               → 10% increase {direction} utilization by {impact_pct:.1f}%")

print(f"\nRobustness Score: {robustness.score:.2f} ({robustness.rating.upper()})")
print(f"Vulnerable Parameters: {', '.join(robustness.vulnerable_parameters)}")

In [ ]:
# Example 3: Constructability Scoring (Ease of construction)
# Score is 0-100 scale: 85+ excellent, 70-84 good, 55-69 acceptable, <55 poor
from structural_lib.insights import calculate_constructability_score
from structural_lib.api import design_beam_is456, detail_beam_is456

# Design a beam (reusing params from above)
design_result = design_beam_is456(**params)

# Get detailing
detailing = detail_beam_is456(
    design_result=design_result,
    span_mm=5000,
    b_mm=params["b_mm"],
    D_mm=params["D_mm"],
    d_mm=params["d_mm"],
    support_condition="SIMPLY_SUPPORTED"
)

# Calculate constructability score
constructability = calculate_constructability_score(
    design_result=design_result,
    detailing=detailing
)

print(f"Constructability Score: {constructability.score:.1f}/100 (Rating: {constructability.rating.upper()})")
print(f"\n{constructability.overall_message}")
print(f"\nFactor Breakdown:")
for factor in constructability.factors:
    sign = "+" if factor.score >= 0 else ""
    print(f"  • {factor.factor}: {sign}{factor.score:.1f} pts")
    print(f"    {factor.message}")
    if factor.recommendation:
        print(f"    💡 {factor.recommendation}")

In [ ]:
# Example 4: CLI with --insights flag (JSON export)
# The --insights flag outputs precheck, sensitivity, and robustness to a separate JSON file

!python -m structural_lib design beams_small.csv -o results_insights.json --insights

# View the insights output
import json
from pathlib import Path

insights_file = Path('results_insights_insights.json')
if insights_file.exists():
    with open(insights_file) as f:
        insights_data = json.load(f)
    print("Insights JSON keys:", list(insights_data.keys()))
    if 'precheck' in insights_data:
        print(f"\nPrecheck risk level: {insights_data['precheck'].get('risk_level', 'N/A')}")
    if 'robustness' in insights_data:
        print(f"Robustness score: {insights_data['robustness'].get('score', 'N/A'):.2f}")
else:
    print("Insights file not generated - run design command first")

### Insights Summary

The insights module helps you:
- **Precheck**: Catch geometry/loading issues before expensive full design (~1ms)
- **Sensitivity**: Understand which parameters most affect your design margin
- **Constructability**: Score ease of construction (0-100) with actionable recommendations

All insights are advisory - the main design always runs. See [Insights API Reference](../reference/insights-api.md) for complete documentation.